In [ ]:
!sudo -E conda install -y tensorflow-gpu keras pandas scipy PyYAML h5py matplotlib Pillow glob2 progressbar2

In [2]:
# Ignore warnings.
import warnings
warnings.filterwarnings("ignore")

# Check python version.
import sys
assert sys.version_info[0] == 3 and sys.version_info[1] == 6, "Requires Python 3.6.x, got " + str(sys.version_info)
print("Python version is fine!")

# Check tensorflow.
import tensorflow
print("tensorflow:", tensorflow.__version__)

# Check keras.
import keras
print("keras:", keras.__version__)

# Check tensorflow.
import matplotlib
print("matplotlib:", matplotlib.__version__)

# Check PIL.
import PIL
print("PIL:", PIL.__version__)

# Check numpy.
import numpy
print("numpy:", numpy.__version__)

# Test datasets.
from keras.datasets import mnist
mnist.load_data()
print("Loaded mnist.")
from keras.datasets import fashion_mnist
fashion_mnist.load_data()
print("Loaded fashion_mnist.")
from keras.datasets import boston_housing
boston_housing.load_data()
print("Loaded boston_housing.")
from keras.datasets import imdb
imdb.load_data()
print("Loaded imdb.")


print("Success!")

Python version is fine!
tensorflow: 1.11.0
keras: 2.2.4
matplotlib: 3.0.0
PIL: 5.3.0
numpy: 1.15.2


Using TensorFlow backend.


11493376/11490434 [==============================] - 5s 0us/step
Loaded mnist.
4423680/4422102 [==============================] - 0s 0us/step
Loaded fashion_mnist.
57344/57026 [==============================] - 0s 3us/step
Loaded boston_housing.
17465344/17464789 [==============================] - 9s 0us/step
Loaded imdb.
Success!


In [3]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 13s 213us/step - loss: 0.2616 - acc: 0.9190 - val_loss: 0.0551 - val_acc: 0.9823
Epoch 2/12
60000/60000 [==============================] - 9s 145us/step - loss: 0.0873 - acc: 0.9735 - val_loss: 0.0420 - val_acc: 0.9860
Epoch 3/12
60000/60000 [==============================] - 9s 146us/step - loss: 0.0655 - acc: 0.9805 - val_loss: 0.0320 - val_acc: 0.9886
Epoch 4/12
60000/60000 [==============================] - 9s 146us/step - loss: 0.0542 - acc: 0.9832 - val_loss: 0.0293 - val_acc: 0.9895
Epoch 5/12
60000/60000 [==============================] - 9s 146us/step - loss: 0.0454 - acc: 0.9861 - val_loss: 0.0273 - val_acc: 0.9905
Epoch 6/12
60000/60000 [==============================] - 9s 145us/step - loss: 0.0383 - acc: 0.9885 - val_loss: 0.0276 - val_acc: 0.9907
Epoch 7/12
60000/60000 [========================

In [5]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print("Available GPUs:", get_available_gpus())

Available GPUs: ['/device:GPU:0']


In [6]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 798203271320623648
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11281927373
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10091231206809581172
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]
